In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import seaborn as sns
import numpy as np
import pandas as pd
import torch
import gc
from datetime import date, timedelta
from torch import nn,optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [2]:
plt.style.use('seaborn-bright')

In [3]:
# If a gpu is available, set gpu else cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Import and Transform the Data

In [4]:
split = '2017-07-15'
def read_process_data(drop_cols):
    df = pd.read_csv('lag.csv',parse_dates=['date'])
    df = df[df['date'] > '2017-6-15']
    df.sort_values(by=['date','store_nbr','family'],inplace=True)
    df.reset_index(drop = True,inplace = True)
    df.drop(drop_cols,axis=1,inplace=True)
    df = create_dummies(df,['store_nbr','family'])
    month = df[df['date'] >= split].to_numpy()
    X_train,y_train,X_test,y_test = data_split(df)
    del df
    gc.collect()
    X_train,y_train,X_test,y_test = data_prep(X_train,y_train,X_test,y_test)
    train_iter,test_iter = loader(X_train,y_train,X_test,y_test)
    return train_iter,test_iter,y_test,month

## Data Visualization

In [5]:
# Defined a function to plot predicted values vs actual values
def plot_prediction(tens):
    #month = data['date'][split:].to_numpy()

    plt.figure(figsize = (18,6))
    plt.plot(month,y_test)
    plt.plot(month,tens.cpu().numpy()[:,-1])
    plt.legend(['Actual Values', 'Predicted Values'], loc='upper left')
    plt.xticks(rotation=90)
    plt.xlabel("Year")
    plt.ylabel("Unit Sales")
    plt.title("Unit Sales vs Year")
    
    ax = plt.gca()
    ax.xaxis.set_major_locator(plticker.MultipleLocator(base=15.0))
    plt.show()

In [6]:
def plot_comp(mse_lst,time_lst):
    fig, ax1 = plt.subplots(figsize = (18,6))

    ax1.set_xlabel('Model Number')
    ax1.set_ylabel('MSE',color='tab:red')
    ax1.plot(list(range(0,len(mse_lst))),np.array(mse_lst), color='tab:red')

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    ax2.set_ylabel('Time (in ms)',color='tab:blue')  # we already handled the x-label with ax1
    ax2.plot(list(range(0,len(mse_lst))),np.array(time_lst), color='tab:blue')
    ax2.tick_params(axis='y')

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()

In [7]:
def create_dummies(data,col_list):
    
    for col_name in col_list:
        
        drop_first = False
        if len(data[col_name].value_counts().index) == 2:
            drop_first = True
            
        tempdf = pd.get_dummies(data[col_name],drop_first = drop_first)
        
        if not drop_first:
            tempdf.columns = [col_name +'_' + str(col) for col in tempdf.columns]
        else:
            tempdf.columns = [col_name]
            
        data = data.drop(col_name,axis = 1).join(tempdf)
        
    return data

In [8]:
def data_split(data):
    # Split data into training and testing sets
    data_train = data[data['date'] < split]
    data_test  = data[data['date'] >= split]

    min_max_scaler = MinMaxScaler()
    
    mm = min_max_scaler.fit(data_train.drop('date',axis = 1))
    data_train = pd.DataFrame(mm.transform(data_train.drop('date',axis = 1)),columns=data_train.drop('date',axis = 1).columns, index=data_train.drop('date',axis = 1).index)
    data_test  = pd.DataFrame(mm.transform(data_test.drop('date',axis = 1)),columns=data_test.drop('date',axis = 1).columns, index=data_test.drop('date',axis = 1).index)
    
    # Split data into dependent and independent variables
    X_train = data_train.drop(['unit_sales'],axis = 1).to_numpy()
    y_train = data_train['unit_sales'].to_numpy()
    X_test  = data_test.drop(['unit_sales'],axis = 1).to_numpy()
    y_test  = data_test['unit_sales'].to_numpy()
    
    return X_train,y_train,X_test,y_test

# Defined a function to convert numpy arrays into tensors
def data_prep(X_train,y_train,X_test,y_test):
    
    # Convert numpy arrays into tensors
    X_train = torch.tensor(X_train).float()
    y_train = torch.tensor(y_train).view(-1, 1).float()

    X_test = torch.tensor(X_test).float()
    y_test = torch.tensor(y_test).view(-1, 1).float()

    return X_train,y_train,X_test,y_test
    

def loader(X_train,y_train,X_test,y_test):
    # Build a data loader object from the two tensors for training data
    train_datasets = torch.utils.data.TensorDataset(X_train, y_train)
    train_iter = torch.utils.data.DataLoader(train_datasets, batch_size=round(len(X_train)/226), shuffle=False)

    # Build a data loader object from the two tensors for testing data
    test_datasets = torch.utils.data.TensorDataset(X_test, y_test)
    test_iter = torch.utils.data.DataLoader(test_datasets, batch_size=round(len(X_test)), shuffle=False)
    
    return train_iter,test_iter

## Backpropagation Through Time

In [9]:
drop_list = ['Oil Price','unit_sales (t-1)','unit_sales (t-2)','unit_sales (t-3)','unit_sales (t-4)',
            'unit_sales (t-5)','unit_sales (t-6)','unit_sales (t-7)','unit_sales (t-8)','unit_sales (t-9)',
            'unit_sales (t-10)','unit_sales (t-11)','unit_sales (t-12)','Oil Price (t-1)','Oil Price (t-2)',
            'Oil Price (t-3)','Oil Price (t-4)','Oil Price (t-5)','Oil Price (t-6)','Oil Price (t-7)',
            'Oil Price (t-8)','Oil Price (t-9)','Oil Price (t-10)','Oil Price (t-11)','Oil Price (t-12)']

train_iter,test_iter,y_test,month = read_process_data(drop_list)

In [10]:
# IDefine a class for the RNN model
class RecurrentModel(nn.Module):
    def __init__(self, input_size, hidden_size, batch_size,num_layers=1):
        super().__init__()
        # Initialize attributes used by the model
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.num_layers = num_layers
        
        # Define an RNN layer
        self.rnn = nn.RNN(self.input_size, self.hidden_size,num_layers = self.num_layers)
    
    # Define a method to set the hidden state at the beginning of every epoch
    def hidden_reset(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_size),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_size))

    def forward(self, x):
        # Define the sequence of operation and pass the input tensor through each operation
        x,_ = self.rnn(x.view(len(x), self.batch_size, -1))
        
        return x[:,:,-1][:,-1].view(len(x),1)

In [11]:
# Define a function to calculate the validation MSE
def test(model):
    # Use no_grad to allow us to perform regular Python operations on tensors, independent of PyTorch’s computation graph
    with torch.no_grad():
        # Loop through the test data
        for inputs, labels in test_iter:
            # Move the tensors to the device selected above (GPU or CPU)
            #inputs = inputs.to(device)
            #labels = labels.to(device)
            # Enter model evaluation mode
            model.to('cpu')
            model.eval()
            # Make predictions on the validation dataset
            y_pred = model(inputs)
            # Calculate the mean squared error for the predictions
            loss = lossfunc(y_pred, labels)
    
    # Return MSE and prediction values
    return loss.item(),y_pred

In [12]:
# Initialze the loss function as Mean Square Error loss
lossfunc = nn.MSELoss()

#Define a function to train the model
def train(model, num_epochs = 200,recurr = False):
    # Initialize the optimizer as Stochastic Gradient Descent and initialize the learning rate
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    i=0
    # Loop over the epochs
    for epoch in range(num_epochs):
        # Check if the model being passed in is a recurrent neural network
        if recurr == True:
            model.hidden = model.hidden_reset()
        # Loop through each batch in the training dataset
        for inputs, labels in train_iter:
            # Move the tensors to the device selected above (GPU or CPU)
            inputs = inputs.to(device)
            labels = labels.to(device)
            # Call the model to perform the forward computation
            outputs = model(inputs)
            # Compute the loss given our outputs and labels
            loss = lossfunc(outputs, labels)
            # Since Gradients are accumulated, we have to zero them
            optimizer.zero_grad()
            # Compute Gradients using backpropogation
            loss.backward()
            # Update weights based on the current gradient
            optimizer.step()
        i += 1
        print(i)

    # Store the final validation and training loss
    test_mse,y_pred = test(model)
    mse = loss.item()
            
    return mse,test_mse,y_pred

In [13]:
rnn_mse_lst = []
rnnt_mse_lst = []
rnny_pred_lst = []
rnn_time = []
for i in [5,18]:
    for j in [2,3]:
        print(f"Model i:{i},j:{j}")
        # Initialize objects to track execution time
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        # Begin recording execution time
        start.record()
        # Initialize model
        model = RecurrentModel(input_size = 1, hidden_size = i, batch_size = 88,num_layers=j)
        model.to(device)
        rnn_mse,rnnt_mse,rnny_pred = train(model,60,True)
        rnn_mse_lst.append(rnn_mse)
        rnnt_mse_lst.append(rnnt_mse)
        rnny_pred_lst.append(rnny_pred)
        # Stop recording execution time
        end.record()
        # Wait for kernels in all streams of a cuda device to complete
        torch.cuda.synchronize()
        # Compute and store execution time
        rnn_time.append(start.elapsed_time(end))

Model i:5,j:2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
Model i:5,j:3
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
Model i:18,j:2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


KeyboardInterrupt: 

In [ ]:
plot_comp(rnnt_mse_lst,rnn_time)

In [ ]:
best_model_rnn = rnnt_mse_lst.index(min(rnnt_mse_lst))
print(f"The best training mean squared error for Backpropogation through time is: {rnn_mse_lst[best_model_rnn]}")
print(f"The best validation mean squared error for Backpropogation through time is: {rnnt_mse_lst[best_model_rnn]}")
print(f"The best computational effort for Backpropogation through time in milliseconds is: {rnn_time[best_model_rnn]}")